In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/articles-sharing-reading-from-cit-deskdrop/shared_articles.csv
/kaggle/input/articles-sharing-reading-from-cit-deskdrop/users_interactions.csv


In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
article = pd.read_csv('/kaggle/input/articles-sharing-reading-from-cit-deskdrop/shared_articles.csv')
user = pd.read_csv('/kaggle/input/articles-sharing-reading-from-cit-deskdrop/users_interactions.csv')

In [4]:
article

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,1487946604,CONTENT SHARED,9213260650272029784,3609194402293569455,7144190892417579456,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,HTML,https://startupi.com.br/2017/02/liga-ventures-...,"Conheça a Liga IoT, plataforma de inovação abe...","A Liga Ventures, aceleradora de startups espec...",pt
3118,1487947067,CONTENT SHARED,-3295913657316686039,6960073744377754728,-8193630595542572738,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,GA,US,HTML,https://thenextweb.com/apps/2017/02/14/amazon-...,Amazon takes on Skype and GoToMeeting with its...,"Amazon has launched Chime, a video conferencin...",en
3119,1488223224,CONTENT SHARED,3618271604906293310,1908339160857512799,-183341653743161643,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0...,SP,BR,HTML,https://code.org/about/2016,Code.org 2016 Annual Report,"February 9, 2017 - We begin each year with a l...",en
3120,1488300719,CONTENT SHARED,6607431762270322325,-1393866732742189886,2367029511384577082,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,HTML,https://www.bloomberg.com/news/articles/2017-0...,JPMorgan Software Does in Seconds What Took La...,"At JPMorgan Chase & Co., a learning machine is...",en


In [5]:
user

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR


In [6]:
article[article['eventType']=='CONTENT REMOVED']

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
59,1459345029,CONTENT REMOVED,7973573994178035769,-8845298781299428018,3760091107461406486,NaN,NaN,NaN,HTML,http://noticias.uol.com.br/politica/ultimas-no...,Governo Dilma é desaprovado por 69% e aprovado...,Pesquisa Ibope encomendada pela Confederação N...,pt
70,1459355695,CONTENT REMOVED,8078873160882064481,3891637997717104548,1776955132850411647,NaN,NaN,NaN,HTML,http://buytaert.net/white-house-deepens-its-co...,Dries Buytaert,White House deepens its commitment to Open Sou...,en
84,1459367793,CONTENT REMOVED,-7356135999773525293,-1443636648652872475,8209530310193218854,NaN,NaN,NaN,HTML,http://www.popsci.com/byzantine-science-deceiv...,Fooling The Machine,"Two groups, one at Berkeley University and ano...",en
87,1459367903,CONTENT REMOVED,7420742904084384944,-1443636648652872475,8209530310193218854,NaN,NaN,NaN,HTML,http://www.popsci.com/byzantine-science-deceiv...,Fooling The Machine,"Two groups, one at Berkeley University and ano...",en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2824,1480942004,CONTENT REMOVED,4169374063813159761,2612012348742830907,-2359713986954060163,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,HTML,http://noticias.uol.com.br/politica/ultimas-no...,"Brasil gasta R$ 16,4 mi ao ano com aposentador...",iStock Uso do cargo para beneficiar loja maçôn...,pt
2874,1481740935,CONTENT REMOVED,8289800212949675494,-8241940599580729220,-4005961876543065182,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR,HTML,http://making.duolingo.com/which-countries-stu...,"Which countries study which languages, and wha...",To find out which languages are the most popul...,en
2942,1484047628,CONTENT REMOVED,-5924653357616257702,-4246040813531967142,-4748637958812360523,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,HTML,http://exame.abril.com.br/tecnologia/robo-da-i...,Robô da IBM substitui 34 funcionários de empre...,"São Paulo - O Watson, que a empresa define com...",pt
3008,1485342148,CONTENT REMOVED,-7358932766266901263,-8420584158427265596,-1843663387254811608,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,HTML,http://www.coindesk.com/three-smart-contract-m...,Why Many Smart Contract Use Cases Are Simply I...,Dr Gideon Greenspan is the founder and CEO of ...,en


In [7]:
a = list(article[article['eventType']=='CONTENT REMOVED']['contentId'])
a

[-6451309518266745024,
 7973573994178035769,
 8078873160882064481,
 -7356135999773525293,
 7420742904084384944,
 8219310215587599928,
 -6499865726739947794,
 -1287202638861144684,
 2293315701090958124,
 6206342468637502921,
 8160885002406274828,
 -4248528062574538011,
 -3265730906936163775,
 2824996248683640175,
 -8099709040471107382,
 -1861959619649725543,
 -8418620743404378592,
 -9153494109165200346,
 -389749390447363927,
 -2220859822082318562,
 756136904399885845,
 7921510628110926276,
 8497861351486449784,
 -2990485643677949494,
 -2990485643677949494,
 -1312850681127865433,
 -4187268503708157132,
 -456574518348806233,
 4739732607073456292,
 -8112397019636931887,
 -1706114177222872702,
 7078276984547732873,
 -6878119266243511199,
 1433951803438448289,
 1556878199027930272,
 -4068545238978420047,
 4315784099325221836,
 -3114161754347804356,
 9033884391004475493,
 -1172724258904585136,
 -4975645025178552956,
 5322532051893153382,
 -8449536481786923901,
 -2309481887656981309,
 82006629

In [8]:
article = article[~article['contentId'].isin(a)].reset_index(drop=True)

In [9]:
article

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
2,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
3,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
4,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978,1487946604,CONTENT SHARED,9213260650272029784,3609194402293569455,7144190892417579456,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,HTML,https://startupi.com.br/2017/02/liga-ventures-...,"Conheça a Liga IoT, plataforma de inovação abe...","A Liga Ventures, aceleradora de startups espec...",pt
2979,1487947067,CONTENT SHARED,-3295913657316686039,6960073744377754728,-8193630595542572738,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,GA,US,HTML,https://thenextweb.com/apps/2017/02/14/amazon-...,Amazon takes on Skype and GoToMeeting with its...,"Amazon has launched Chime, a video conferencin...",en
2980,1488223224,CONTENT SHARED,3618271604906293310,1908339160857512799,-183341653743161643,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0...,SP,BR,HTML,https://code.org/about/2016,Code.org 2016 Annual Report,"February 9, 2017 - We begin each year with a l...",en
2981,1488300719,CONTENT SHARED,6607431762270322325,-1393866732742189886,2367029511384577082,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,HTML,https://www.bloomberg.com/news/articles/2017-0...,JPMorgan Software Does in Seconds What Took La...,"At JPMorgan Chase & Co., a learning machine is...",en


In [10]:
article = article[article['lang']=='en'].reset_index(drop=True)

In [11]:
article

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
2,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
3,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
4,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2161,1487939756,CONTENT SHARED,4675505028897335428,-1393866732742189886,-1729556941184852519,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,HTML,https://medium.mybridge.co/swift-top-10-articl...,Swift Top 10 Articles For The Past Year (v.2017),"For the past year , we've ranked nearly 9,000 ...",en
2162,1487947067,CONTENT SHARED,-3295913657316686039,6960073744377754728,-8193630595542572738,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,GA,US,HTML,https://thenextweb.com/apps/2017/02/14/amazon-...,Amazon takes on Skype and GoToMeeting with its...,"Amazon has launched Chime, a video conferencin...",en
2163,1488223224,CONTENT SHARED,3618271604906293310,1908339160857512799,-183341653743161643,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0...,SP,BR,HTML,https://code.org/about/2016,Code.org 2016 Annual Report,"February 9, 2017 - We begin each year with a l...",en
2164,1488300719,CONTENT SHARED,6607431762270322325,-1393866732742189886,2367029511384577082,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,HTML,https://www.bloomberg.com/news/articles/2017-0...,JPMorgan Software Does in Seconds What Took La...,"At JPMorgan Chase & Co., a learning machine is...",en


In [12]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 1.0, 
   'BOOKMARK': 1.5, 
   'FOLLOW': 2.0,
   'COMMENT CREATED': 1.5,  
}

user['eventStrength'] = user['eventType'].apply(lambda x: event_type_strength[x])

In [13]:
user

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,2.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


In [14]:
user.loc[(user['personId']==-9223121837663643404) & (user['contentId']==-7423191370472335463)]

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
60988,1478687166,VIEW,-7423191370472335463,-9223121837663643404,2411828781515959462,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
61256,1478795892,VIEW,-7423191370472335463,-9223121837663643404,-4096620279830729916,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
63155,1478860184,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
63180,1478862151,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
63183,1478860897,VIEW,-7423191370472335463,-9223121837663643404,844431470588928965,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
63834,1479376578,VIEW,-7423191370472335463,-9223121837663643404,-6084152871682607649,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
64003,1478705793,VIEW,-7423191370472335463,-9223121837663643404,2411828781515959462,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
65167,1479290925,VIEW,-7423191370472335463,-9223121837663643404,-8865295409778928525,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0


In [15]:
user = user.drop_duplicates(subset=['personId', 'contentId', 'eventType'], keep='last').reset_index(drop=True)

In [16]:
user.loc[(user['personId']==-9223121837663643404) & (user['contentId']==-7423191370472335463)]

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
45198,1479290925,VIEW,-7423191370472335463,-9223121837663643404,-8865295409778928525,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0


In [17]:
users_interactions_count_df = user.groupby(['personId', 'contentId', 'sessionId']).size().groupby(['personId', 'contentId']).size().reset_index()
users_interactions_count_df = users_interactions_count_df.rename(columns={0: "count"})
print('# users: %d' % len(users_interactions_count_df))

# users: 40710


In [18]:
users_interactions_count_df

,personId,contentId,count
0,-9223121837663643404,-8949113594875411859,1
1,-9223121837663643404,-8377626164558006982,1
2,-9223121837663643404,-8208801367848627943,1
3,-9223121837663643404,-8187220755213888616,1
4,-9223121837663643404,-7423191370472335463,1
...,...,...,...
40705,9210530975708218054,8477804012624580461,1
40706,9210530975708218054,8526042588044002101,1
40707,9210530975708218054,8856169137131817223,1
40708,9210530975708218054,8869347744613364434,1


In [19]:
users_interactions_count_df.loc[users_interactions_count_df['personId']==-9212075797126931087]

,personId,contentId,count
43,-9212075797126931087,-1995591062742965408,1
44,-9212075797126931087,-969155230116728853,1
45,-9212075797126931087,1796618361228441836,1
46,-9212075797126931087,5162508842577612756,1
47,-9212075797126931087,6852597772196653540,1


In [20]:
users_interactions_count_df.loc[users_interactions_count_df['personId']==-9223121837663643404]

,personId,contentId,count
0,-9223121837663643404,-8949113594875411859,1
1,-9223121837663643404,-8377626164558006982,1
2,-9223121837663643404,-8208801367848627943,1
3,-9223121837663643404,-8187220755213888616,1
4,-9223121837663643404,-7423191370472335463,1
5,-9223121837663643404,-7331393944609614247,1
6,-9223121837663643404,-6872546942144599345,1
7,-9223121837663643404,-6728844082024523434,1
8,-9223121837663643404,-6590819806697898649,1
9,-9223121837663643404,-6558712014192834002,1


In [21]:
users_articles_count_df = users_interactions_count_df.groupby(['personId', 'contentId']).size().groupby(['personId']).size().reset_index()
users_articles_count_df = users_articles_count_df.rename(columns={0: "count"})
users_articles_count_df = users_articles_count_df.loc[users_articles_count_df['count']>=5].reset_index(drop=True)
print('# users articles counts: %d' % len(users_articles_count_df))

# users articles counts: 1140


In [22]:
users_articles_count_df

,personId,count
0,-9223121837663643404,43
1,-9212075797126931087,5
2,-9207251133131336884,7
3,-9199575329909162940,11
4,-9196668942822132778,7
...,...,...
1135,9165571805999894845,5
1136,9187866633451383747,17
1137,9191849144618614467,12
1138,9199170757466086545,14


In [23]:
print('# of interactions: %d' % len(user))
interactions_from_selected_users_df = user.merge(users_articles_count_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 50938
# of interactions from users with at least 5 interactions: 49119


In [24]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum().reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 39106


,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.0
1,-9223121837663643404,-8377626164558006982,1.0
2,-9223121837663643404,-8208801367848627943,1.0
3,-9223121837663643404,-8187220755213888616,1.0
4,-9223121837663643404,-7423191370472335463,1.0
5,-9223121837663643404,-7331393944609614247,1.0
6,-9223121837663643404,-6872546942144599345,1.0
7,-9223121837663643404,-6728844082024523434,1.0
8,-9223121837663643404,-6590819806697898649,1.0
9,-9223121837663643404,-6558712014192834002,1.0


In [25]:
interactions_full_df

,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.0
1,-9223121837663643404,-8377626164558006982,1.0
2,-9223121837663643404,-8208801367848627943,1.0
3,-9223121837663643404,-8187220755213888616,1.0
4,-9223121837663643404,-7423191370472335463,1.0
...,...,...,...
39101,9210530975708218054,8477804012624580461,5.5
39102,9210530975708218054,8526042588044002101,1.0
39103,9210530975708218054,8856169137131817223,1.0
39104,9210530975708218054,8869347744613364434,1.0


In [26]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 31284
# interactions on Test set: 7822


In [27]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [28]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [29]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(article['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()   

In [30]:
def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

In [31]:
interactions_test_indexed_df

,contentId,eventStrength
personId,,
-830175562779396891,5928346445655989915,1.0
-7267769888748948232,2708089973817733462,2.0
-3535274684588209118,-1564468014116140117,1.0
3094513233385472738,-5011709367955107239,1.0
-3390049372067052505,-6255158415883847921,2.0
...,...,...
-5639303336519418445,4158328739563138055,1.0
-6874532444478397764,5745373608367143062,1.0
-6003416708481128042,7516228655554309785,2.0


In [32]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('contentId')['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,contentId,eventStrength
0,-4029704725707465084,296.5
1,-6783772548752091658,239.5
2,-133139342397538859,210.5
3,8224860111193157980,195.0
4,-8208801367848627943,193.0
5,-2358756719610361882,188.5
6,-6843047699859121724,178.0
7,7507067965574797372,169.0
8,2581138407738454418,167.0
9,1469580151036142903,166.0


In [33]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['eventStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, article)

In [34]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
1139 users processed

Global metrics:
{'modelName': 'Popularity', 'recall@5': 0.28228074661211966, 'recall@10': 0.4102531321912554}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,37,61,192,0.192708,0.317708,3609194402293569455
17,16,28,134,0.119403,0.208955,-2626634673110551643
16,15,29,130,0.115385,0.223077,-1032019229384696495
10,4,9,117,0.034188,0.076923,-1443636648652872475
82,22,40,88,0.250000,0.454545,-2979881261169775358
161,13,21,80,0.162500,0.262500,-3596626804281480007
65,26,35,73,0.356164,0.479452,1116121227607581999
81,18,23,69,0.260870,0.333333,692689608292948411
106,16,24,69,0.231884,0.347826,-9016528795238256703
52,27,33,68,0.397059,0.485294,3636910968448833585


In [35]:
#Ignoring stopwords (words with no semantics) from English
stopwords_list = stopwords.words('english')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = article['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(article['title'] + "" + article['text'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<2166x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 546884 stored elements in Compressed Sparse Row format>

In [36]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    try:
        len(ids)
    except Exception:
        ids = [ids]
        pass
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['contentId'] \
                                                   .isin(article['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [37]:
user_profiles = build_users_profiles()
len(user_profiles)

1114

In [38]:
myprofile = user_profiles[-1479311724257856983]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[-1479311724257856983].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 5000)


,token,relevance
0,learning,0.292075
1,machine learning,0.261628
2,machine,0.244238
3,google,0.167301
4,data,0.165870
5,ai,0.161117
6,algorithms,0.112322
7,search,0.087613
8,people,0.081392
9,language,0.079601


In [39]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [40]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        try:
            similar_items = self._get_similar_items_to_user_profile(user_id)
        except:
            similar_items = []
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(article)

In [41]:
for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
    print(idx, person_id)

0 -830175562779396891
1 -7267769888748948232
2 -3535274684588209118
3 3094513233385472738
4 -3390049372067052505
5 -3500661007957156229
6 -8399605302938582500
7 -7990997793599977496
8 3681539658619684640
9 5328649570833345873
10 -1443636648652872475
11 -1251984896177895077
12 -2901997243166418276
13 7585672106741478592
14 7410250256723888301
15 1874422396201148365
16 -1032019229384696495
17 -2626634673110551643
18 -4951114616026800382
19 -9223121837663643404
20 1899224753659027997
21 6247767575778104994
22 8698209668239173600
23 8759620187357735368
24 -9109785559521267180
25 2901546026085255870
26 1766257854965201953
27 3915038251784681624
28 -5661737290289697460
29 -2184225127953983407
30 495422936350381071
31 6756039155228175109
32 6686977580828209471
33 -1130272294246983140
34 -3196433790714486571
35 -8020832670974472349
36 -3896187702811204367
37 5497189205340943824
38 4432373331608915019
39 -4448388750105189364
40 -9009798162809551896
41 3576137684812235192
42 7392990465409599343


In [42]:
for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
    try:
        content_based_recommender_model._get_similar_items_to_user_profile(person_id)
    except:
        print(idx, person_id)
        pass

284 4063400901401648079
291 -5374802517306837266
357 7555752854236030786
531 -5732707497022383920
551 -1272364595235252747
598 7072373334511597461
616 6773859371279096371
782 -2644571257397324782
809 -6820774667818944570
847 3138250281946918035
855 -6874849957985527250
893 -468849228680532284
895 8791271631167250981
900 5758100189335708355
913 7048619884024295715
915 -5833749125541124532
917 6648370769037295216
991 -5388083602161519863
995 -9196668942822132778
1003 -8306418824693202277
1016 -2826197582088460753
1032 8494103216511135887
1077 -7980653669913930599
1083 -2116515004529086881
1100 -7920036110689552764
1101 2530443148414180019


In [43]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
1139 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.1102019943748402, 'recall@10': 0.17335719764766044}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,14,17,192,0.072917,0.088542,3609194402293569455
17,16,25,134,0.119403,0.186567,-2626634673110551643
16,15,26,130,0.115385,0.200000,-1032019229384696495
10,24,34,117,0.205128,0.290598,-1443636648652872475
82,4,9,88,0.045455,0.102273,-2979881261169775358
161,10,16,80,0.125000,0.200000,-3596626804281480007
65,5,10,73,0.068493,0.136986,1116121227607581999
81,5,14,69,0.072464,0.202899,692689608292948411
106,4,8,69,0.057971,0.115942,-9016528795238256703
52,1,7,68,0.014706,0.102941,3636910968448833585


In [44]:
user.loc[user['personId']==-1479311724257856983]

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
20,1465414324,VIEW,4306804750094230211,-1479311724257856983,5326691377159417610,NaN,NaN,NaN,1.0
339,1461688626,VIEW,8637400873589298078,-1479311724257856983,-6854178205345850546,Android - Native Mobile App,SP,BR,1.0
456,1459776444,VIEW,6987435400166905688,-1479311724257856983,1649981580224206877,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
461,1459776502,BOOKMARK,6987435400166905688,-1479311724257856983,1649981580224206877,NaN,NaN,NaN,1.5
466,1459777106,VIEW,8049518593334748695,-1479311724257856983,1649981580224206877,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
...,...,...,...,...,...,...,...,...,...
45490,1482370943,VIEW,1580124381109623727,-1479311724257856983,-3818662099051238898,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
46567,1485899335,FOLLOW,-532999578436827210,-1479311724257856983,-7587005918600298385,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,2.0
46568,1485899335,COMMENT CREATED,-532999578436827210,-1479311724257856983,-7587005918600298385,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.5
48339,1485952422,VIEW,-532999578436827210,-1479311724257856983,907648221007678235,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0


In [45]:
content_based_recommender_model._get_similar_items_to_user_profile(-1479311724257856983)

[(5250363310227021277, 0.6439401624316128),
 (-7126520323752764957, 0.639301122468413),
 (638282658987724754, 0.6143932365864088),
 (5258604889412591249, 0.5843790530357534),
 (2220561310072186802, 0.5641388756538991),
 (-8068727428160395745, 0.5589128799491665),
 (-229081393244987789, 0.5573906443119695),
 (54678605145828343, 0.5293460793258352),
 (-4571929941432664145, 0.5260365229598271),
 (-9128652074338368262, 0.5175054821910792),
 (-9033211547111606164, 0.5023474590020723),
 (-7702672626132856079, 0.5006398375201009),
 (5092635400707338872, 0.4873160095846136),
 (1328618437884612347, 0.4824118260623697),
 (7342707578347442862, 0.4803758334930629),
 (3269302169678465882, 0.475138939692075),
 (-6244754482621048226, 0.47058251798682793),
 (-8123434787655959885, 0.46746138606451715),
 (-6940659689413147290, 0.4667834427727317),
 (201515581783532281, 0.4664255591659721),
 (3564394485543941353, 0.46480518660274067),
 (2589533162305407436, 0.4621379039282404),
 (-3745582971859460684, 0.